In [15]:
import scapy.all
import struct
import socket

packet_list = scapy.all.rdpcap('../data/pcap/eq1.pcap', count=1000)

for pkt in packet_list:
    if 'IP' in pkt and 'TCP' in pkt:
        break
else:
    raise 'no tcp packets'

pkt

<IP  version=4 ihl=5 tos=0x0 len=52 id=61140 flags=DF frag=0 ttl=48 proto=tcp chksum=0x8fcc src=81.124.179.179 dst=43.36.155.207 |<TCP  sport=58353 dport=http seq=1134734331 ack=567200089 dataofs=8 reserved=0 flags=A window=412 chksum=0x8dd3 urgptr=0 options=[('NOP', None), ('NOP', None), ('Timestamp', (21465, 1253162392))] |>>

In [13]:
print(bytes(pkt))
pkt.show()

b'E\x00\x004\xee\xd4@\x000\x06\x8f\xccQ|\xb3\xb3+$\x9b\xcf\xe3\xf1\x00PC\xa2\xab\xfb!\xce\xc9Y\x80\x10\x01\x9c\x8d\xd3\x00\x00\x01\x01\x08\n\x00\x00S\xd9J\xb1\xbd\x98'
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 52
  id        = 61140
  flags     = DF
  frag      = 0
  ttl       = 48
  proto     = tcp
  chksum    = 0x8fcc
  src       = 81.124.179.179
  dst       = 43.36.155.207
  \options   \
###[ TCP ]### 
     sport     = 58353
     dport     = http
     seq       = 1134734331
     ack       = 567200089
     dataofs   = 8
     reserved  = 0
     flags     = A
     window    = 412
     chksum    = 0x8dd3
     urgptr    = 0
     options   = [('NOP', None), ('NOP', None), ('Timestamp', (21465, 1253162392))]



In [27]:
b = bytes(pkt)
to_sub = struct.pack("!4s4sHH",
    scapy.all.inet_pton(socket.AF_INET, pkt['IP'].src),
    scapy.all.inet_pton(socket.AF_INET, pkt['IP'].dst),
    6,
    pkt['IP'].len - pkt.ihl * 4
) + bytes(pkt['TCP'])[:16] + bytes(pkt['TCP'])[18:20]

to_sub2 = bytes(pkt['IP'])[12:20] + struct.pack('!HH', 6, pkt['IP'].len - pkt.ihl * 4) + bytes(pkt['TCP'])[:16] + bytes(pkt['TCP'])[18:20]

print(len(to_sub))
print(to_sub == to_sub2)

for b in to_sub:
    print(hex(b))

30
True
0x51
0x7c
0xb3
0xb3
0x2b
0x24
0x9b
0xcf
0x0
0x6
0x0
0x20
0xe3
0xf1
0x0
0x50
0x43
0xa2
0xab
0xfb
0x21
0xce
0xc9
0x59
0x80
0x10
0x1
0x9c
0x0
0x0


In [58]:
import array

if struct.pack("H", 1) == b"\x00\x01":  # big endian
    checksum_endian_transform = lambda chk: chk  # type: Callable[[int], int]
else:
    checksum_endian_transform = lambda chk: ((chk >> 8) & 0xff) | chk << 8

def checksum(data):
    if len(data) % 2 == 1:
        data += b"\0"

    s = sum(array.array("H", data))
    s = (s >> 16) + (s & 0xffff)
    s += s >> 16
    s = ~s
    return checksum_endian_transform(s) & 0xffff

def remove_from_checksum(c, data):
    # undo the inversions for endian and one's complement
    c = checksum_endian_transform(c)
    c = ~c & 0xffff

    # subtract the crc of the data, apply a correction for the number of wraps past 0
    #for word in array.array("H", pseudo):
    #    c -= word
    #    if c < 0:
    #        c += 2**16 - 2
    c -= sum(array.array("H", data))

    # re-apply the one's complement
    c = (c >> 16) + (c & 0xffff)
    c += c >> 16
    c = ~c
    return checksum_endian_transform(c) & 0xffff

In [59]:
for pkt in packet_list:
    if 'IP' not in pkt:
        continue

    for proto in ['TCP', 'UDP']:
        if proto in pkt:
            break
    else:
        continue

    if len(bytes(pkt)) != pkt['IP'].len:
#        print('truncated packet', proto, len(bytes(pkt)), pkt['IP'].len)
        continue

    ib = bytes(pkt['IP'])
    pb = bytes(pkt[proto])
    ilen = pkt['IP'].len
    plen = ilen - pkt.ihl * 4
    # IPV4 pseudo header: src, dst, proto, tcp segment length
    # TCP pseudo header: TCP header without checksum
    pseudo = ib[12:20] + struct.pack('!HH', pkt['IP'].proto, plen)
    
    if proto == 'TCP':
        pseudo += pb[0:16] + pb[18:20]
    elif proto == 'UDP':
        pseudo += pb[0:6]

    reversed_csum = remove_from_checksum(pkt[proto].chksum, pseudo)

    payload = pb[20 if proto == 'TCP' else 8:]
    payload_csum = checksum(payload)

    print(proto, reversed_csum, payload_csum)
    if reversed_csum != payload_csum:
        break


TCP 39633 39633
TCP 62889 62889
TCP 53236 53236
TCP 33723 33723
TCP 21990 21990
TCP 30084 30084
TCP 30084 30084
TCP 43398 43398
TCP 43398 43398
TCP 43398 43398
TCP 65535 65535
TCP 43398 43398
TCP 65535 65535
TCP 39076 39076
TCP 65535 65535
TCP 53373 53373
TCP 21990 21990
TCP 14562 14562
TCP 39479 39479
TCP 65535 65535
TCP 65535 65535
TCP 23193 23193
TCP 65535 65535
TCP 15705 15705
TCP 26421 26421
TCP 5931 5931
TCP 65535 65535
TCP 53185 53185
TCP 65535 65535
TCP 48704 48704
TCP 9265 9265
TCP 61287 61287
TCP 10475 10475
TCP 1719 1719
TCP 64154 64154
TCP 20385 20385
TCP 30167 30167
TCP 1095 1095
TCP 61247 61247
TCP 30167 30167
TCP 32274 32274
TCP 22314 22314
TCP 56243 56243
TCP 5405 5405
TCP 4233 4233
TCP 65535 65535
TCP 65535 65535
TCP 59704 59704
TCP 65535 65535
TCP 31794 31794
TCP 52043 52043
TCP 9245 9245
TCP 45372 45372
TCP 65535 65535
TCP 58088 58088
TCP 2619 2619
TCP 2619 2619
TCP 10475 10475
TCP 26247 26247
TCP 65535 65535
TCP 26247 26247
TCP 26247 26247
TCP 10475 10475
TCP 65535 